In [ ]:
import numpy as np
from autograd import grad, elementwise_grad
import autograd.numpy.random as npr
import matplotlib.pyplot as plt
from net_visualiser import DrawNN

## The Problem
We want to solve the following BVP $$y'' = -2,\quad0<x<1$$ $$y(0) = y(1) = 1$$ using a neural network. This BVP has analytic solution $$y = 1 + x(1-x)$$ which we will use to evaluate our solution. We first define the loss function we shall use.

## Backpropagation
In order to train our neural network, we use gradient descent. This is simple repeated calculation of the form $$\theta_{n+1} = \theta_n-\alpha\nabla_\theta L$$ where $L = L(x_k,\theta)$ is the loss function and $\theta$ are the parameters. How do we calculate $\nabla_\theta L$? For a network of $N$ layers, this vector is given $$\nabla_\theta L = \left(\frac{\partial L}{\partial W^{(1)}},\dots,\frac{\partial L}{\partial W^{(N)}}, \frac{\partial L}{\partial b^{(1)}},\dots,\frac{\partial L}{\partial b^{(N)}}\right)$$

In [ ]:
def initialise_params(architecture): # will generalise later
    W1 = np.random.randn(architecture[1], architecture[0])* 0.01
    b1 = np.random.randn(architecture[1], 1)
    return [W1, b1]

def neural_network(params,x, architecture):
    params = initialise_params(architecture)

def y_trial(x, params):
    return x*(1 - x)*neural_network(x,params) + 1

def y(x_k, params,activ_function = sigmoid):
    W = params[0]
    b = params[1]
    return activ_function(W@x_k + b)


def loss(x_k, params):
    dy = elementwise_grad(y,0)
    ddy = elementwise_grad(dy,0)(x_k, params)
    return np.sum((ddy + 2*np.ones_like(ddy))**2)

In [ ]:
def analytic(x):
    return 1 + x*(1-x)

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def initialise_params(x): # will generalise later
    n = len(x)
    W1 = np.random.randn(n, n)#* 0.01
    b1 = np.random.randn(n, 1)
    return [W1, b1]

def loss(params, x,activ_function = sigmoid):
    # unpack
    W = params[0]
    b = np.reshape(params[1],(len(x),))
    # single layer
    y_hat = activ_function(W@x + b)
    ## preprocess
    left_bc_vec = np.zeros_like(x)
    left_bc_vec[0] = 1 - y_hat[0]#._value
    right_bc_vec = np.zeros_like(x)
    right_bc_vec[-1] = 1 - y_hat[-1]#._value
    y = lambda x: y_hat + (np.ones_like(x)-x)*left_bc_vec + x*right_bc_vec
    dy = elementwise_grad(y)
    ddy = elementwise_grad(dy,0)(x)
    return np.sum((ddy + 2*np.ones_like(ddy))**2)

def solve_ode_neural_network(x, epochs, learning_rate, activ_function=sigmoid,):
    costs = []
    ## Initialise Parameters
    params = initialise_params(x)
    for e in range(epochs):
        # print(f'Epoch:{e}')
        W = params[0]
        b = params[1]
        ## backpropagation
        loss_grad = grad(loss)

        ## Compute loss
        cost = loss(params, x)
        if e % 100 == 0 or i == epochs:
            costs.append(cost)
        for i in range(100):
            # Compute gradients
            gradients = loss_grad(params, x)
            dW, db = gradients

        # Update parameters using gradient descent
            params[0] -= learning_rate * dW
            params[1] -= learning_rate * db
    plt.plot(np.squeeze(costs))
    return params

def net_approximation(x, final_params, activ_function = sigmoid):
    W = final_params[0]
    b = np.reshape(final_params[1],(len(x),))
    return activ_function(W@x + b)

In [ ]:
## Approximation
N = 50
epochs = 1000
learning_rate = 0.001
x = np.linspace(0,1,N)
params = solve_ode_neural_network(x,epochs, learning_rate)
y_n = net_approximation(x, params)

In [ ]:
# Paramters
fig, ax = plt.subplots(1,2,figsize = (12,7))
ax[0].plot(np.linspace(0,1,1000),analytic(np.linspace(0,1,1000)),label = 'Analytic Solution')
ax[0].plot(x,y_n,label = 'Approximation')
ax[0].set_xlabel('x')
ax[0].set_ylabel('y(x)')
ax[0].set_title('Solution')
ax[0].grid()
ax[0].legend()
ax[1].set_title('Error')
# final error, NN loss 

In [ ]:

def train(x_k, architecture, activ_function, epochs, learning_rate):
    params = initialise_params(architecture)
    for _ in range(epochs):
        W = params[0]
        b = params[1]
        
        loss_func_grad = grad(loss,0)
        ## Backward prop
        for i in range(100): # make parameter
            cost_grad =  loss_func_grad(x_k, params)
            params[0] = params[0] - learning_rate * cost_grad[0]
            params[1] = params[1] - learning_rate * cost_grad[1]
    return params

def net_approximation(x_k, final_params):
    return y(x_k,final_params)

In [ ]:
DrawNN(architecture).draw()

In [ ]:

def train(x_k, architecture, activ_function, epochs, learning_rate):
    ## Intialise Params
    params = initialise_params(architecture)
    ## Begin Training 
    for _ in range(epochs):
        
        ## Forward Prop - make forward prop function
        y_k = single_layer_network(x_k, activ_function, params)

        ## Post process
        # y = post_process(y_k, x_k)
        W = params[0]
        b = params[1]
        def y(x_k):
            y_k = activ_function(W@x_k + b)
            return y_k + (1-x_k)*(y_k[0] - 1) + x_k*(y_k[-1] - 1)
        
        ## Compute derivatives
        y_x = elementwise_grad(y)
        y_xx = elementwise_grad(y_x)(x_k)
        
        ## Calculate Loss
        L = loss(y_xx,x_k)
        loss_func_grad = grad(loss,0)
        ## Backward prop
        for i in range(100): # make parameter
            cost_grad =  loss_func_grad(params, x_k)
            params[0] = params[0] - learning_rate * cost_grad[0]
            params[1] = params[1] - learning_rate * cost_grad[1]

        ## Backward prop
    return params